In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from scipy.spatial import distance as dist
from imutils import face_utils
import numpy as np
import dlib
import cv2
import pyautogui as m
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
import winsound

m.FAILSAFE = False




top = Tk()
top.title("OTS")
top.minsize(600, 600)



Font_tuple = ("Comic Sans MS", 8, "bold")
Font_tuple1 = ("Comic Sans MS",24, "bold")

number1label = Label(text="OTS",fg="blue",font=("Comic Sans MS",43, "bold"))
number1label.pack()


#open webcam

def Open():

    #taking size of screen
    (scrx,scry)=m.size()

    mLocOld = np.array([0,0])
    mouseLoc = np.array([0,0])
    DampingFactor = 15          # supposed to make mouse smooth

    def calculateView(x,y):
        xvMax, yvMax = m.size()
        xvMin, yvMin = 0, 0
        xwMax, xwMin = 370, 270
        ywMax, ywMin = 290, 200
        sx = (xvMax - 0) // (xwMax - xwMin)
        sy = (yvMax - 0) // (ywMax - ywMin)
        xv = xvMin + (x - xwMin) * sx
        yv = yvMin + (y - ywMin) * sy
        return xv,yv

    def eye_aspect_ratio(eye):
        # compute the euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])

        # compute the euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])

        # compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)

        # return the eye aspect ratio
        return ear

    # define two constants, one for the eye aspect ratio to indicate
    # blink and then a second constant for the number of consecutive
    # frames the eye must be below the threshold
    PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
    EYE_AR_THRESH = .15
    EYE_AR_CONSEC_FRAMES = 7
    # initialize the frame counters and the total number of blinks
    COUNTER = 0
    TOTAL = 0
    # initialize dlib's face detector (HOG-based) and then create
    # the facial landmark predictor
    #print("[INFO] loading facial landmark predictor...")
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(PREDICTOR_PATH)
    # grab the indexes of the facial landmarks for the left and
    # right eye, respectively
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    (nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
    # start the video stream thread
    #print("[INFO] starting video stream thread...")
    from pynput.mouse import Button, Controller
    mouse = Controller()
    mouse.position = (750,350)
    vs = cv2.VideoCapture(0)
    
    ######################################################################################
    detector1 = FaceMeshDetector(maxFaces=1)
    plotY = LivePlot(640, 360, [20, 50], invert=True)
 
    idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
    ratioList = []
    blinkCounter = 0
    counter = 0
    color = (255, 0, 255)

    # loop over frames from the video stream
    while True:
        # grab the frame from the threaded video file stream, resize
        # it, and convert it to grayscale
        # channels)
        ret, frame = vs.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detect faces in the grayscale frame
        rects = detector(gray, 0)

            # loop over the face detections
        for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            nose = shape[nStart:nEnd]
            rightEye = shape[rStart:rEnd]

            #print('nose')
            #print(nose[0])

            # get window coordinates

            xv, yv = nose[0]

            xw = np.int(xv)
            yw = np.int(yv)
            #print(type(xv))
            xv,yv = calculateView(xw,yw)

            # for mouse control        
            mouseLoc = mLocOld + ((xv,yv)-mLocOld)//DampingFactor
            #print('nx = {} and ny = {}'.format(mouseLoc[0], mouseLoc[1]))
            m.moveTo(mouseLoc[0],mouseLoc[1],tween =  m.linear(.5))

            # calculate the eye-aspect-ratio(EAR) for both eyes
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            #cv2.drawContours(frame, [noseHull], -1, (0, 255, 0), 1)

            #if rightEAR < .15:
             #   m.click(mouseLoc[0],mouseLoc[1],clicks = 1, button = 'left' )

            #if leftEAR < .15:
             #   m.click(mouseLoc[0],mouseLoc[1],clicks = 1, button = 'right')

            mLocOld = mouseLoc
            
            ###############################################################
        #blink
        success, img = vs.read()
        img, faces = detector1.findFaceMesh(img, draw=False)
 
        if faces:
            face = faces[0]
            leftUp = face[159]
            leftDown = face[23]
            leftLeft = face[130]
            leftRight = face[243]
            lenghtVer, _ = detector1.findDistance(leftUp, leftDown)
            lenghtHor, _ = detector1.findDistance(leftLeft, leftRight)
 
 
            ratio = int((lenghtVer / lenghtHor) * 100)
            ratioList.append(ratio)
            if len(ratioList) > 3:
                ratioList.pop(0)
            ratioAvg = sum(ratioList) / len(ratioList)

            if ratioAvg < 29 and counter == 0:
                blinkCounter += 1
                m.click(clicks=2)
                winsound.Beep(500, 250)
                counter = 1
            if counter != 0:
                counter += 1
                if counter > 10:
                    counter = 0
                    color = (255,0, 255)
        
        

        # show the frame
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1)==13:
            break

       
    vs.release()
    cv2.destroyAllWindows()

############################
but1 = Button(text="Start", command=Open,bg="pink",fg="white",font=Font_tuple1)
but1.pack(side = TOP,expand = True )
top.mainloop()

C:\Users\EL10_gazy\AppData\Local\Temp\ipykernel_14920\75649842.py:140: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xw = np.int(xv)
C:\Users\EL10_gazy\AppData\Local\Temp\ipykernel_14920\75649842.py:141: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for ad

In [2]:
import tkinter